In [10]:
import subprocess
import re
from pathlib import Path
import time

In [66]:
# === Configuration ===
run_numbers = [str(i) for i in range(234095, 234149)]

# - paths -
data_path = "/xustrg0/2025A8039"
process_path = "/UserData/andronis/test_data/"
output_path = process_path+"processed"
mask_path = "/home/myeongsik/mask_234110.npy"
poni_file = "/home/seoyoung/2025_05_SACLA/results/avg_img/run234059_avg_img.poni"

# -- optional arguments -- 
nbins = 250  # number of q-bins
n_phi = 36   # number of phi-bins
n_chunks = 10  # total number of chunks

In [67]:
for run_number in run_numbers:
    subprocess.run([
        "python", "process.py",
        "--data_path", data_path,
        "--run_number", run_number,
        "--mask_path", mask_path,
        "--output_path", output_path,
        "--poni_file", poni_file,
        "--nbins", str(nbins),
        "--n_phi", str(n_phi),
        "--n_chunks", str(n_chunks)
    ])


Array job submitted with ID: 8489505
Merge job submitted successfully with job ID: 8489506.fep01
Array job submitted with ID: 8489507
Merge job submitted successfully with job ID: 8489508.fep01
Array job submitted with ID: 8489509
Merge job submitted successfully with job ID: 8489510.fep01
Array job submitted with ID: 8489511
Merge job submitted successfully with job ID: 8489512.fep01
Array job submitted with ID: 8489513
Merge job submitted successfully with job ID: 8489514.fep01
Array job submitted with ID: 8489515
Merge job submitted successfully with job ID: 8489516.fep01
Array job submitted with ID: 8489517
Merge job submitted successfully with job ID: 8489518.fep01
Array job submitted with ID: 8489519
Merge job submitted successfully with job ID: 8489520.fep01
Array job submitted with ID: 8489521
Merge job submitted successfully with job ID: 8489522.fep01
Array job submitted with ID: 8489523
Merge job submitted successfully with job ID: 8489524.fep01
Array job submitted with ID: 8

In [71]:
# === Check job status ===
!qstat -u andronis


fep01: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
8489591[].fep01 andronis serial   iq_chunk      --    1   1    4gb 00:30 B   -- 
8489592.fep01   andronis serial   combine_c*    --    1   1    4gb 00:10 H   -- 
8489595[].fep01 andronis serial   iq_chunk      --    1   1    4gb 00:30 B   -- 
8489596.fep01   andronis serial   combine_c*    --    1   1    4gb 00:10 H   -- 
8489597[].fep01 andronis serial   iq_chunk      --    1   1    4gb 00:30 B   -- 
8489598.fep01   andronis serial   combine_c*    --    1   1    4gb 00:10 H   -- 
8489599[].fep01 andronis serial   iq_chunk      --    1   1    4gb 00:30 Q   -- 
8489600.fep01   andronis serial   combine_c*    --    1   1    4gb 00:10 H   -- 
8489601[].fep01 andronis serial   iq_chunk      --    1   1    4gb 00:30 Q   -- 
8489602.fep01   andro

In [73]:
!ls -lh /UserData/andronis/test_data/processed/

total 18G
-rw-r--r-- 1 andronis girelli  69M Jun 11 17:53 Iq_234095_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 17:56 Iq_234096_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 17:56 Iq_234097_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 17:58 Iq_234098_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 17:59 Iq_234099_combined.h5
-rw-r--r-- 1 andronis girelli 282M Jun 11 17:59 Iq_234100_combined.h5
-rw-r--r-- 1 andronis girelli  12M Jun 11 17:58 Iq_234101_combined.h5
-rw-r--r-- 1 andronis girelli  60M Jun 11 17:59 Iq_234102_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 18:01 Iq_234103_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 18:02 Iq_234104_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 18:03 Iq_234105_combined.h5
-rw-r--r-- 1 andronis girelli 412M Jun 11 18:04 Iq_234106_combined.h5
-rw-r--r-- 1 andronis girelli  43M Jun 11 18:03 Iq_234107_combined.h5
-rw-r--r-- 1 andronis girelli  92M Jun 11 18:05 Iq_234108_combined.h5
-rw-r--r--

rm: cannot remove '/UserData/girelli/sacla2024/processed/processed/': Is a directory
